In [5]:
import numpy as np
import pandas as pd
import matplotlib
pd.set_option("display.precision", 4)

from internalForces import BendingMoment, ShearForce
from loadMatrix import LoadMatrix
from memberStiffnessMatrix import MemberStiffnessMatrix
from transformationMatrices import rotationMatrix, transmissionMatrix
from elasticModulus import elasticModulus
from momentOfInertia import momentOfInertia

In [6]:
def parseMatrixRows(matrix, indexes):
    index = {}
    for i in indexes:
        index.update({indexes.index(i): i})
    newMatrix = matrix.reset_index(drop=True).rename(index=index)

In [7]:
def parseMatrixColumns(matrix,i):
    matrix.columns = i
    return matrix

In [8]:
def parseMatrixIndexes(matrix, i):
    x = parseMatrixColumns(matrix,i)
    return parseMatrixRows(x,i)

## Vstupy

In [9]:
# Section properties
width = 200 #mm
height = 300 #mm
concrete = "C30/37"
# -------------------------
E = elasticModulus(concrete)
I = momentOfInertia(width, height)
AREA = width*height*10**-6
print("E = ", E, "kPa")
print("I = ", I, "m-4")
print("A = ", AREA, "m2")

E =  33000000.0 kPa
I =  0.00044999999999999993 m-4
A =  0.06 m2


In [11]:
# lengths
A = 3 #m
B = 3 #m
C = 3 #m

# loads
F = 40 #kN
Q = 5 #kN/m
T = 40 #kN

## Vzorové riešenie na prúte 1-2

In [12]:
k12 = MemberStiffnessMatrix.localMember(C,AREA,E,I)
k12

,0,1,2
0,660000.0,0.0,0.0
1,0.0,6600.0,9900.0
2,0.0,9900.0,19800.0


In [13]:
alfa = 270 #deg
A0_1 = rotationMatrix(alfa)
A0_1

,0,1,2
0,-1.8370e-16,-1.0000e+00,0
1,1.0000e+00,-1.8370e-16,0
2,0.0000e+00,0.0000e+00,1
